<a href="https://colab.research.google.com/github/broshanfekr/GLCN/blob/main/GLCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 1.x

In [ ]:
!mkdir -p data/cora/
!wget -O data.zip https://www.dropbox.com/s/4lwcel37kpihzh8/cora.zip?dl=0
!unzip data.zip -d /content/data/cora

In [21]:
import numpy as np
import scipy.sparse as sp
import sys
import scipy.io as sio
import tensorflow as tf
import time
import os

**مقدار دهی برخی از پارامترهای مربوط به آموزش مدل**

In [22]:
class Flags:
  def __init__(self):
    self.dataset = 'cora'
    self.model = 'sglcn'
    self.lr1 = 0.005
    self.lr2 = 0.005
    self.epochs = 10000
    self.hidden_gcn = 30
    self.hidden_gl = 70
    self.dropout = 0.6
    self.weight_decay = 1e-4
    self.early_stopping = 100
    self.losslr1 = 0.01
    self.losslr2 = 0.0001
    self.seed = 123

FLAGS = Flags()

**آماده سازی دیتاست جهت آموزش مدل**

In [23]:
def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def load_data(dataset_str, path="data/"):
    path = path + dataset_str + "/"
    if dataset_str == "cora":
        features = sio.loadmat(path + "feature")
        features = features['matrix']
        adj = sio.loadmat(path + "adj")
        adj = adj['matrix']
        labels = sio.loadmat(path + "label")
        labels = labels['matrix']
        idx_train = range(140)
        idx_val = range(200, 500)
        idx_test = range(500, 1500)
    elif dataset_str == "citeseer":
        features = sio.loadmat(path + "feature")
        features = features['matrix']
        adj = sio.loadmat(path + "adj")
        adj = adj['matrix']
        labels = sio.loadmat(path + "label")
        labels = labels['matrix']
        idx_test = sio.loadmat(path + "test.mat")
        idx_test = idx_test['array'].flatten()
        idx_train = range(120)
        idx_val = range(120, 620)
    else:
        features = sio.loadmat(path + "feature")
        features = features['matrix']
        adj = sio.loadmat(path + "adj")
        adj = adj['matrix']
        labels = sio.loadmat(path + "label")
        labels = labels['matrix']
        idx_test = sio.loadmat(path + "test.mat")
        idx_test = idx_test['matrix']
        idx_train = range(60)
        idx_val = range(200, 500)

    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask

**پیش پردازش ویژگی ها و همچنین پیش پردازش و نرمال سازی ماتریس مجاورت**

In [24]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)


def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()


def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    edge = np.array(np.nonzero(adj_normalized.todense()))
    return sparse_to_tuple(adj_normalized), edge

**مقداردهی اولیه به وزن های شبکه عصبی و همچنین ایجاد ساختمان داده مربوط به ورودی شبکه عصبی**

In [25]:
def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)
    

def construct_feed_dict(features, adj, labels, labels_mask, epoch, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels})
    feed_dict.update({placeholders['labels_mask']: labels_mask})
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['adj']: adj})
    feed_dict.update({placeholders['step']: epoch})
    feed_dict.update({placeholders['num_nodes']: features[2][0]})
    feed_dict.update({placeholders['num_features_nonzero']: features[1].shape})
    return feed_dict

**محاسبه معیارهای ارزیابی**

In [26]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

**تعریف لایه کانولوشن گرافی و همچنین لایه مربوط به یادگیری ساختار گراف**

In [27]:
# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)


def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res


class SparseGraphLearn(object):
    """Sparse Graph learning layer."""
    def __init__(self, input_dim, output_dim, edge, placeholders, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.num_nodes = placeholders['num_nodes']
        self.sparse_inputs = sparse_inputs
        self.bias = bias
        self.edge = edge

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = glorot([input_dim, output_dim], name='weights')
            self.vars['a'] = glorot([output_dim, 1], name='a')
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

    def __call__(self, inputs):
        x = inputs
        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # graph learning
        h = dot(x, self.vars['weights'], sparse=self.sparse_inputs)
        N = self.num_nodes
        edge_v = tf.abs(tf.gather(h,self.edge[0]) - tf.gather(h,self.edge[1]))
        edge_v = tf.squeeze(self.act(dot(edge_v, self.vars['a'])))
        sgraph = tf.SparseTensor(indices=tf.transpose(self.edge), values=edge_v, dense_shape=[N, N])
        sgraph = tf.sparse_softmax(sgraph)
        return h, sgraph


class GraphConvolution(object):
    """Graph convolution layer provided by Thomas N. Kipf, Max Welling, 
    [Semi-Supervised Classification with Graph Convolutional Networks]
    (http://arxiv.org/abs/1609.02907) (ICLR 2017)"""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.sparse_inputs = sparse_inputs
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = glorot([input_dim, output_dim], name='weights')
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs, adj):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # convolve
        pre_sup = dot(x, self.vars['weights'], sparse=self.sparse_inputs)
        output = dot(adj, pre_sup, sparse=True)

        # bias
        if self.bias:
            output += self.vars['bias']

        return self.act(output)

    def __call__(self, inputs, adj):
        with tf.name_scope(self.name):
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs, adj)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])


**ساخت مدل نهایی**

In [28]:
class SGLCN(object):
    def __init__(self, placeholders, edge, input_dim, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging
        self.loss1 = 0
        self.loss2 = 0

        self.inputs = placeholders['features']
        self.edge = edge
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        learning_rate1 = tf.train.exponential_decay(learning_rate=FLAGS.lr1, global_step=placeholders['step'],
                                                    decay_steps=100, decay_rate=0.9, staircase=True)
        learning_rate2 = tf.train.exponential_decay(learning_rate=FLAGS.lr2, global_step=placeholders['step'],
                                                    decay_steps=100, decay_rate=0.9, staircase=True)
        self.optimizer1 = tf.train.AdamOptimizer(learning_rate=learning_rate1)
        self.optimizer2 = tf.train.AdamOptimizer(learning_rate=learning_rate2)

        self.layers0 = SparseGraphLearn(input_dim=self.input_dim,
                                        output_dim=FLAGS.hidden_gl,
                                        edge=self.edge,
                                        placeholders=self.placeholders,
                                        act=tf.nn.relu,
                                        dropout=True,
                                        sparse_inputs=True)

        self.layers1 = GraphConvolution(input_dim=self.input_dim,
                                        output_dim=FLAGS.hidden_gcn,
                                        placeholders=self.placeholders,
                                        act=tf.nn.relu,
                                        dropout=True,
                                        sparse_inputs=True,
                                        logging=self.logging)

        self.layers2 = GraphConvolution(input_dim=FLAGS.hidden_gcn,
                                        output_dim=self.output_dim,
                                        placeholders=self.placeholders,
                                        act=lambda x: x,
                                        dropout=True,
                                        logging=self.logging)
        self.build()
        self.pro = tf.nn.softmax(self.outputs)

    def _loss(self):
        # Weight decay loss
        for var in self.layers0.vars.values():
            self.loss1 += FLAGS.weight_decay * tf.nn.l2_loss(var)
        for var in self.layers1.vars.values():
            self.loss2 += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Graph Learning loss
        D = tf.matrix_diag(tf.ones(self.placeholders['num_nodes']))*-1
        D = tf.sparse_add(D, self.S)*-1
        D = tf.matmul(tf.transpose(self.x), D)
        self.loss1 += tf.trace(tf.matmul(D, self.x)) * FLAGS.losslr1
        self.loss1 -= tf.trace(tf.sparse_tensor_dense_matmul(tf.sparse_transpose(self.S), tf.sparse_tensor_to_dense(self.S))) * FLAGS.losslr2

        # Cross entropy error
        self.loss2 += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])
       
        self.loss = self.loss1 + self.loss2

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def build(self):
        self.x, self.S = self.layers0(self.inputs)

        x1 = self.layers1(self.inputs, self.S)
        self.outputs = self.layers2(x1, self.S)

        # Store model variables for easy access
        self.vars1 = tf.trainable_variables()[0:2]
        self.vars2 = tf.trainable_variables()[2:]

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op1 = self.optimizer1.minimize(self.loss1, var_list=self.vars1)
        self.opt_op2 = self.optimizer2.minimize(self.loss2, var_list=self.vars2)
        self.opt_op = tf.group(self.opt_op1, self.opt_op2)

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [29]:
# Define model evaluation function
def evaluate(features, adj, labels, mask, epoch, placeholders,flag=0):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, adj, labels, mask, epoch, placeholders)
    if flag == 0:
        outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
        return outs_val[0], outs_val[1], (time.time() - t_test)
    else:
        outs_val = sess.run(model.accuracy, feed_dict=feed_dict_val)
        return outs_val

np.random.seed(FLAGS.seed)
tf.set_random_seed(FLAGS.seed)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

# Some preprocessing
features = preprocess_features(features)
adj, edge = preprocess_adj(adj)

# Define placeholders
placeholders = {
    'adj': tf.sparse_placeholder(tf.float32),
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_nodes': tf.placeholder(tf.int32),
    'step': tf.placeholder(tf.int32),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = SGLCN(placeholders, edge, input_dim=features[2][1], logging=True)

# Initialize session
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto()  
config.gpu_options.per_process_gpu_memory_fraction = 1
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
# sess = tf.Session()

# Init variables
sess.run(tf.global_variables_initializer())
test_acc_list = []
best_epoch = 0
best = 10000

# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, adj, y_train, train_mask, epoch, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})
    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)
    # Validation
    cost, acc, duration = evaluate(features, adj, y_val, val_mask, epoch, placeholders)
    test_acc = evaluate(features, adj, y_test, test_mask, epoch, placeholders, flag=1)
    test_acc_list.append(test_acc)


    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "test_acc=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(time.time() - t))

    if cost < best:
        best_epoch = epoch
        best = cost
        patience = 0

    else:
        patience += 1

    if patience == FLAGS.early_stopping:
        # feed_dict_val = construct_feed_dict(features, adj, y_test, test_mask, epoch, placeholders)
        # Smap = sess.run(tf.sparse_tensor_to_dense(model.S), feed_dict=feed_dict_val)
        # sio.savemat("S.mat", {'adjfix': np.array(Smap)})
        break
 
print("Optimization Finished!")
print("----------------------------------------------")
print("The finall result:", test_acc_list[-101])
print("----------------------------------------------")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 0001 train_loss= 2.19247 train_acc= 0.21429 val_loss= 1.96447 val_acc= 0.26000 test_acc= 0.30400 time= 1.05668
Epoch: 0002 train_loss= 2.11923 train_acc= 0.40000 val_loss= 1.94037 val_acc= 0.46333 test_acc= 0.43100 time= 0.36667
Epoch: 0003 train_loss= 2.06540 train_acc= 0.50714 val_loss= 1.91946 val_acc= 0.54333 test_acc= 0.47400 time= 0.36518
Epoch: 0004 train_loss= 2.02314 train_acc= 0.55000 val_loss= 1.90130 val_acc= 0.58333 test_acc= 0.47800 time= 0.35680
Epoch: 0005 train_loss= 1.98001 train_acc= 0.57143 val_loss= 1.88563 val_acc= 0.59000 test_acc= 0.48000 time= 0.36787
Epoch: 0006 train_loss= 1.94517 train_acc= 0.60714 val_loss= 1.87210 val_acc= 0.58667 test_acc= 0.47500 